In [ ]:
import pandas as pd
df= pd.read_csv(
    "/path/to/ckpt_100000_pt_0_001_vs_Stockfish_1_adv.csv"
)
df

In [ ]:
for i, row in df.iterrows():
    break

In [ ]:
import json
sampled_moves = json.loads(row['all_sampled_moves'])
sampled_moves
for ply in sampled_moves:
    if ply is None:
        continue
    for k, v in ply.items():
        tuple_everything = [] #original does not have tuples for none
        for i in v:
            if i is None:
                tuple_everything.append((None, None))
            else:
                tuple_everything.append(i)
        
        ply[k] = tuple_everything

sampled_moves

In [ ]:
import torch
import numpy as np
stats = torch.load('./stats_3.pt')


# %pip install scipy
from scipy.stats import bootstrap

test = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
k = bootstrap(test[None], np.mean)
# print(k.standard_error * 1.96, np.mean(stats[i]) - k.confidence_interval.low)
# call bootstrap function

def calc_stat(stat):
    mean =  np.mean(np.array(stat))
    bootstrap_ci = bootstrap(stat[None], np.mean)
    # print(round(mean, 4), bootstrap_ci.confidence_interval)
    return round(mean, 4), mean - bootstrap_ci.confidence_interval.low

for i in stats:
    if not isinstance(stats[i][0], dict):   
        stats[i] = np.array(stats[i]) * 100
        # print(i, round(np.mean(np.array(stats[i])), 4))
        print(i, calc_stat(np.array(stats[i])))
    else:
        print()
        print(i)
        for k in stats[i][0]:
            print(k, calc_stat(np.array([j[k] for j in stats[i]]) * 100))
            # print(k, round(np.mean(np.array([j[k] for j in stats[i]])) * 100, 4))
        print()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams.update({"font.size": 13})

# use tex

plt.rcParams.update(
    {
        "text.usetex": True,
        "text.latex.preamble": r"\usepackage{amsfonts}",
    }
)


# plt.yscale("log", base=2)
# plt.xscale("symlog", base=1.0)
plt.figure(figsize=(8, 2.15), dpi=300)

ax = plt.gca()
# ax.set_xlim([-.1, 0.1])
ax.set_xlim([-10, 10])

COLORS = ["green", "red"]

for i, k in enumerate(['acc_deltas_0.001', 'acc_deltas_0.75']):
    # for i, k in enumerate(["advantage_deltas_0.001", "advantage_deltas_0.5"]):
    counts, bins = np.histogram(stats[k], bins=100, density=True, range=(-20, 20))
    # counts, bins = np.histogram(stats[k], bins=50, density=True)
    plt.hist(
        bins[:-1],
        bins=bins,
        weights=counts,
        density=True,
        histtype="bar",
        label=fr"$f'=f_{{\tau={k.split('_')[-1]}}}$",
        color=COLORS[i],
        alpha=0.2,
    )
    mean = np.mean(stats[k])
    # plt.axvline(x=mean, color=COLORS[i],                
    #             label=rf"$\mathbb{{E}}[F]:$ {mean:.2f}",
    #             )

#
plt.axvline(x=0, color="black", label=rf"Baseline $\mathbb{{E}}[F]:$ 0.0")
# plt.text(
#     # mean - 0.4175,
#     0.07,
#     plt.ylim()[1] * (0.75 - 0.2 * 2),
#     rf"Baseline $\mathbb{{E}}[F]:$ 0.0",
#     color="black",
#     verticalalignment="top",
#     weight="bold",
# )

for i, k in enumerate(
    [
        "acc_deltas_0.001",
        "acc_deltas_0.75",
    ]
):
    mean = np.mean(stats[k])
    plt.axvline(
        x=mean,
        color=COLORS[i],
        label=rf"$\mathbb{{E}}[F]:$ {mean:.2f}",
    )
plt.legend()
plt.tight_layout()
plt.xlabel(r"Change in expected reward over base $f_{\tau=1}$ probability distribution over individual states: $F(f', f)$")
plt.ylabel(r"$\mathbb{P}(F)$")

In [ ]:
counts, bins = np.histogram(stats["advantage_deltas_0.5"], bins=50)
plt.hist(bins[:-1], bins, weights=counts, density=True)